In [1]:
#import packages
import pandas as pd
import numpy as np


# 1. Load fake dataframes

In [2]:
# read fake dataframes 

df_users = pd.read_csv('fake_dataframes/fake_df_users.csv', index_col = 0)
df_users_rank = pd.read_csv('fake_dataframes/fake_df_users_rank.csv', index_col = 0)
df_score = pd.read_csv('fake_dataframes/fake_df_score.csv', index_col = 0)
df_organ_external_data = pd.read_csv('fake_dataframes/fake_df_organ_external_data.csv', index_col = 0)
df_organization = pd.read_csv('fake_dataframes/fake_df_organization.csv', index_col = 0)
scoring_cols = pd.read_csv('fake_dataframes/fake_scoring_cols.csv', index_col = 0)
weights = pd.read_csv('fake_dataframes/fake_weights.csv', index_col = 0)

# 2. Data processing

### 2.1 Organizations

In [3]:
# set default values if that organziation is not in our "df_organ_external_data"

uid_list = []
oname_list = []
active_regions_list = []
age_list = []
community_outreach_list = []
consumer_rating_list = []
employee_diversity_list = []
financial_situation_list = []
media_coverage_list = []
number_of_employee_list = []
office_locations_list = []
revenue_list = []
social_media_activity_list = []
stock_performance_list = []
total_assets_list = []
website_list = []

##
international_student_employee_rate_list = []
employee_average_salary_list = []
retention_rate_list = []
employee_benefits_list = []
##

In [4]:
# check the uid whether is in the "df_organ_external_data" or not 
# formulate lists of metrics 

for u in df_organization['uid'].unique():
    if u not in df_organ_external_data['uid'].values:
        uid_list.append(u)
        oname_list.append(df_organization.loc[df_organization['uid']==u,'website'].values[0])
        active_regions_list.append(1)
        age_list.append(1)
        community_outreach_list.append(1)
        consumer_rating_list.append(1)
        employee_diversity_list.append(1)
        financial_situation_list.append(1)
        media_coverage_list.append(1)
        number_of_employee_list.append(1)
        office_locations_list.append(1)
        revenue_list.append(1)
        social_media_activity_list.append(1)
        stock_performance_list.append(0)
        total_assets_list.append(1)
        website_list.append(1)
        international_student_employee_rate_list.append(0)
        employee_average_salary_list.append(30000)  #set a defalut salary $30,000
        retention_rate_list.append(0.5) #set a defalut retention rate $0.5
        employee_benefits_list.append(2.5) #set a defalut employee_benefits 2.5
        

In [9]:
# cast existing lists of metrics to temporary dataframe 

temp_data = pd.DataFrame({'uid': uid_list,'oname': oname_list, 'active_regions': active_regions_list, 
                          'age': age_list, 'community_outreach': community_outreach_list, 
                          'international_student_employee_rate': international_student_employee_rate_list,
                          'consumer_rating': consumer_rating_list, 'employee_diversity': employee_diversity_list, 
                          'financial_situation': financial_situation_list, 
                          'employee_average_salary': employee_average_salary_list,'retention_rate': retention_rate_list,
                          'employee_benefits': employee_benefits_list, 'media_coverage': media_coverage_list, 
                          'number_of_employee': number_of_employee_list, 'office_locations': office_locations_list, 
                          'revenue': revenue_list, 'social_media_activity': social_media_activity_list, 
                          'stock_performance': stock_performance_list, 'total_assets': total_assets_list, 
                          'website': website_list}).fillna(0)

# merge the temporary dataframe with df_organ_external_data
df_organ_external_data = pd.concat([df_organ_external_data, temp_data], axis=0, sort=False, ignore_index=True)

In [10]:
df_organ_external_data.head()

,uid,oname,active_regions,age,community_outreach,international_student_employee_rate,consumer_rating,employee_diversity,financial_situation,employee_average_salary,retention_rate,employee_benefits,media_coverage,number_of_employee,office_locations,revenue,social_media_activity,stock_performance,total_assets,website
0,1286.0,hfui,8.0,141.0,1.0,0.257,4.3,1.0,4.3478,58409.0,0.97,4.2,17000000.0,44000.0,1.0,4.000000e+09,158000.0,0.00,9.200000e+09,1.0
1,1285.0,wnqy,1.0,85.0,1.0,0.000,4.5,1.0,3.3000,104099.0,0.80,4.0,596000.0,17000.0,1.0,7.000000e+09,3300000.0,0.00,2.124100e+10,1.0
2,1287.0,lsyq,152.0,98.0,1.0,0.000,4.1,1.0,3.2443,93454.0,0.74,4.0,256000000.0,203000.0,1.0,6.538800e+10,7581839.0,184.84,2.020000e+11,1.0


### 2.2 Employee 

In [20]:
# get the data of that organization's employee

uid_list = []
oname_list = []
num_individ_from_company_list = []
sum_individ_impact_score_list = []
mean_individ_impact_score_list = []
number_of_upload_list = []
sum_of_the_number_of_followers_list = []

for ids in df_organ_external_data['uid']:
    name = df_organ_external_data.loc[df_organ_external_data['uid']==ids, 'oname'].values[0]
    # find the organization's empolyees by searching email key word
    uids = df_users[df_users['email'].astype(str).str.contains(name)]['uid']
    # get these empolyees' individual impact score
    member_info = df_users_rank.loc[df_users_rank['uid'].isin(uids),:]
    
    uid_list.append(ids)
    oname_list.append(name)
    num_individ_from_company_list.append(member_info['uid'].nunique())
    sum_individ_impact_score_list.append(round(member_info['score'].sum()))
    mean_individ_impact_score_list.append(round(member_info['score'].sum()/member_info['uid'].nunique(),2))
    number_of_upload_list.append(len(df_organization[(df_organization['service_type'].isin(['job_posting', 'marketing_posting'])) & 
                (df_organization['website'].astype(str).str.contains(name))]))
    sum_of_the_number_of_followers_list.append(len(df_organization[(df_organization['service_type'].isin(['company_member'])) & 
                (df_organization['website'].astype(str).str.contains(name))]))

    
# Assemble all data into a dataframe    
internal_data = pd.DataFrame({'uid': uid_list,'oname': oname_list, 
                              'number_of_individual_from_the_company': num_individ_from_company_list, 
                              'sum_of_the_individuals_impact_scores': sum_individ_impact_score_list, 
                              'mean_of_the_individuals_impact_scores': mean_individ_impact_score_list,
                              'number_of_upload': number_of_upload_list, 
                              'sum_of_the_number_of_followers': sum_of_the_number_of_followers_list}).fillna(0)


In [21]:
# set default values for the metrics' data that are not availble now  
columns_list = ['number_of_self_organization_comment', 'number_of_self_organization_follower',
       'number_of_self_organization_rating', 'sum_of_the_number_of_comment',
       'sum_of_the_number_of_competition_completed', 'sum_of_the_number_of_views',
       'sum_of_the_rating_score']

# if the columns from columns_list are not exist in internal_data, we create that columns and insert a default value
for i in columns_list:
    if i not in internal_data:
        internal_data[i] = 1

In [22]:
# calculate the basic mean or proportion date
internal_data['mean_of_the_number_of_views']=round(internal_data['sum_of_the_number_of_views']/(internal_data['number_of_upload']+1),2)
internal_data['mean_of_the_rating_score']=round(internal_data['sum_of_the_rating_score']/(internal_data['number_of_upload']+1),2)
internal_data['mean_of_the_number_of_comment']=round(internal_data['sum_of_the_number_of_comment']/(internal_data['number_of_upload']+1),2)
internal_data['proportion_of_the_company_employees_involved']=round(internal_data['number_of_individual_from_the_company']/df_organ_external_data['number_of_employee'],3)

In [23]:
# merge external data and internal data
internal_data=internal_data.drop(['oname'], axis=1)
new_df = pd.merge(left=df_organ_external_data, right=internal_data, on='uid', how='inner')

# clean the column names
new_df.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_") \
              .replace("$","").replace("%","").replace("#","number") for x in new_df.columns]

# convert column "international_student_employee_rate" to float
new_df['international_student_employee_rate'] = new_df['international_student_employee_rate'].astype(float)

# final version of input date
new_df

,uid,oname,active_regions,age,community_outreach,international_student_employee_rate,consumer_rating,employee_diversity,financial_situation,employee_average_salary,retention_rate,employee_benefits,media_coverage,number_of_employee,office_locations,revenue,social_media_activity,stock_performance,total_assets,website,number_of_individual_from_the_company,sum_of_the_individuals_impact_scores,mean_of_the_individuals_impact_scores,number_of_upload,sum_of_the_number_of_followers,number_of_self_organization_comment,number_of_self_organization_follower,number_of_self_organization_rating,sum_of_the_number_of_comment,sum_of_the_number_of_competition_completed,sum_of_the_number_of_views,sum_of_the_rating_score,mean_of_the_number_of_views,mean_of_the_rating_score,mean_of_the_number_of_comment,proportion_of_the_company_employees_involved
0,1286,usc,8,141,1,0.257,4.3,1,4.3478,58409,0.97,4.2,17000000,44000,1,4.000000e+09,158000,0.00,9.200000e+09,1,187,322.0,1.72,2,1,1,1,1,1,1,1,1,0.33,0.33,0.33,0.004
1,1285,nasa,1,85,1,0.000,4.5,1,3.3000,104099,0.80,4.0,596000,17000,1,7.000000e+09,3300000,0.00,2.124100e+10,1,8,12.0,1.54,2,1,1,1,1,1,1,1,1,0.33,0.33,0.33,0.000
2,1287,disney,152,98,1,0.000,4.1,1,3.2443,93454,0.74,4.0,256000000,203000,1,6.538800e+10,7581839,184.84,2.020000e+11,1,0,0.0,0.00,2,0,1,1,1,1,1,1,1,0.33,0.33,0.33,0.000
3,1067,https://rmdslab.com,1,1,1,0.000,1.0,1,1.0000,30000,0.50,2.5,1,1,1,1.000000e+00,1,0.00,1.000000e+00,1,0,0.0,0.00,0,1,1,1,1,1,1,1,1,1.00,1.00,1.00,0.000
4,1079,grmds.org,1,1,1,0.000,1.0,1,1.0000,30000,0.50,2.5,1,1,1,1.000000e+00,1,0.00,1.000000e+00,1,2,14.0,7.08,3,5,1,1,1,1,1,1,1,0.25,0.25,0.25,2.000
5,1091,www.abc.com,1,1,1,0.000,1.0,1,1.0000,30000,0.50,2.5,1,1,1,1.000000e+00,1,0.00,1.000000e+00,1,0,0.0,0.00,0,2,1,1,1,1,1,1,1,1.00,1.00,1.00,0.000
6,1193,www.grmds.org,1,1,1,0.000,1.0,1,1.0000,30000,0.50,2.5,1,1,1,1.000000e+00,1,0.00,1.000000e+00,1,0,0.0,0.00,0,2,1,1,1,1,1,1,1,1.00,1.00,1.00,0.000
7,1288,https://www.starbucks.com/,1,1,1,0.000,1.0,1,1.0000,30000,0.50,2.5,1,1,1,1.000000e+00,1,0.00,1.000000e+00,1,0,0.0,0.00,2,1,1,1,1,1,1,1,1,0.33,0.33,0.33,0.000


# 3. Calculation 

In [24]:
# convert the some columns data type from object to float64 for calcuation
# scoring_cols
scoring_cols['a'] = scoring_cols['a'].astype('float')
scoring_cols['c'] = scoring_cols['c'].astype('float')
scoring_cols['k'] = scoring_cols['k'].astype('float')
scoring_cols['upper_bound'] = scoring_cols['upper_bound'].astype('float')

# weights
weights['category_weight'] = weights['category_weight'].astype('float')
weights['metric_weight'] = weights['metric_weight'].astype('float')

##  3.1 Calculation function

In [25]:
def score_calculation(scoring_cols, weights):
    
    ############Calculation part01!! (use cols_calculation table)############
    user_score=new_df['uid']
    for index, row in scoring_cols.iterrows():
        score=pd.DataFrame()
        column=row['metrics']
        calc_type=row['function']
        upper=row['upper_bound']
        k=row['k']
        output='score_'+str(column)
        isc_col = new_df[column]


        if calc_type == 'Log':
            z=-k*isc_col
            #print(z)
            z = z.astype(float)
            score[output] = 200/(1+np.exp(z))-100
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, isc_col], axis=1, sort=False)
        elif calc_type == 'Linear':
            score[output] = isc_col.apply(lambda x: (100*x/upper) if (100*x/upper) < 100 else 100)
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, isc_col], axis=1, sort=False)
        elif calc_type == 'Boolean':
            # change the bolean
            score[output] = isc_col.apply(lambda x: 0 if x == 0 else 100)
            #print(score[output])
            user_score = pd.concat([user_score, score], axis=1, sort=False)
            user_score = pd.concat([user_score, isc_col], axis=1, sort=False)
    
    ################ Calculation part02!! (use weights table)####################
    df_user=user_score
    score_cols = [col for col in df_user.columns if 'score_' in col]
    score_cols = ['uid'] + score_cols
    df_user = df_user[score_cols]
    df_user = df_user.fillna(0)


    # prepare empty dictionaries
    dimension_to_metric_dict = dict()
    dimension_weight_dict = dict()
    metric_weight_dict = dict()


    # use the wieght table from our database

    for row in weights.values:
        dimension_weight_dict[row[1]] = float(row[2])
        metric_weight_dict[row[0]] = float(row[3])
        if row[1] in dimension_to_metric_dict:
            dimension_to_metric_dict[row[1]].append(row[0])
        else:
            dimension_to_metric_dict[row[1]] = []
            dimension_to_metric_dict[row[1]].append(row[0])

    for dimension in dimension_to_metric_dict:
        df_user[dimension] = np.zeros(len(df_user))

    for dimension in dimension_to_metric_dict:
        for metric in dimension_to_metric_dict[dimension]:
            if metric in user_score.columns:
                df_user[dimension] += df_user[metric] * metric_weight_dict[metric]

    df_user["total_score"] = np.zeros(len(df_user))
    for dimension in dimension_to_metric_dict:
        df_user["total_score"] += df_user[dimension] * dimension_weight_dict[dimension]
    
    ##### main calculation is done, next make the rank #######
    # standardize columns name
    df_user.columns = [x.lower().replace(" ", "").replace("-","_").replace(r"/","_").replace("\\","_") \
              .replace("$","").replace("%","").replace("#","number") for x in df_user.columns]
    # rank
    df_user = df_user.sort_values(by=["total_score"], ascending=False)
    df_user['rank'] = df_user['total_score'].rank(method='min',ascending=False)
    df_user = df_user[['uid','projects_products_impact','perceived_capacity','perceived_participation','social_and_environmental_impact','employee_individual_impact_score','organization_impact_score','organization_and_employee_interaction','total_score','rank']]
    df_user.columns = ['uid','projects_products_impact','perceived_capacity','perceived_participation','social_and_environmental_impact','employee_individual_impact_score','organization_impact_score','organization_and_employee_interaction','total_score','rank']
    
    return df_user

In [26]:
final_result = score_calculation(scoring_cols, weights)
final_result

,uid,projects_products_impact,perceived_capacity,perceived_participation,social_and_environmental_impact,employee_individual_impact_score,organization_impact_score,organization_and_employee_interaction,total_score,rank
0,1286,88.656124,91.886487,75.796790,95.14,60.821368,6.664570,8.968165,52.418114,1.0
2,1287,93.700000,97.223391,100.000000,90.00,0.000000,6.380117,8.968165,40.936683,2.0
1,1285,70.447478,92.998766,66.253958,90.00,13.829965,6.644570,8.968165,37.772859,3.0
4,1079,10.947487,36.827222,20.000116,90.00,21.904485,13.740368,8.968165,21.519413,4.0
5,1091,10.947487,36.827222,20.000116,90.00,0.000000,7.252983,8.968165,14.705364,5.0
6,1193,10.947487,36.827222,20.000116,90.00,0.000000,7.252983,8.968165,14.705364,5.0
3,1067,10.947487,36.827222,20.000116,90.00,0.000000,6.990006,8.968165,14.642250,7.0
7,1288,10.947487,36.827222,20.000116,90.00,0.000000,6.644570,8.968165,14.559345,8.0
